## Implementation Section

In [1]:
import pandas as pd

df_auc =pd.read_csv("df_auction_to2024.csv",sep=";")
df_auc.head()

,date,logistieke location id,crop_type,fpc_index,cropgroupcode_coded,productgroupcode_coded,country of origin iso,sales value id,total_transactions,total_items_sold,total_package,total_revenue,avg_price_per_item
0,2021-05-25,3,BL,fpc_23,19,45,NL,5,218,16860,493.0,10994.70,0.65
1,2021-02-05,3,BL,fpc_27,122,211,NL,5,1557,1847980,12404.0,388688.80,0.21
2,2021-08-02,3,BL,fpc_23,32,67,KE,5,79,47125,656.0,10694.75,0.23
3,2021-05-14,3,BL,fpc_23,41,80,NL,5,236,38680,627.0,18026.90,0.47
4,2021-03-12,1,BL,fpc_24,11,27,NL,5,499,20424,1673.0,17598.28,0.86


In [2]:
df_dir =pd.read_csv("df_direct_to2024.csv",sep=";")
df_dir.head()

,date,logistieke location id,crop_type,fpc_index,cropgroupcode_coded,productgroupcode_coded,country of origin iso,sales value id,total_transactions,total_items_sold,total_package,total_revenue,avg_price_per_item
0,2021-11-10,NaN,PL,fpc_8,319,480,NL,4,432,16463,1291.0,43189.32,2.62
1,2021-03-29,NaN,PL,fpc_7,600,824,NL,4,246,3847,2637.0,63031.20,16.38
2,2022-07-11,NaN,BL,fpc_6,119,193,NL,4,80,108390,991.0,26821.05,0.25
3,2021-01-26,NaN,BL,fpc_13,6,15,NL,4,5066,1192334,13934.0,195380.07,0.16
4,2022-11-16,NaN,PL,fpc_18,380,564,NL,4,231,8004,1042.0,16564.46,2.07


In [3]:
df_fust =pd.read_csv("df_fust_to2024.csv",sep=";")
df_fust.head()

,Date,fustcode,number_received,number_issued
0,2021-01-02,Fc153,0.0,6.0
1,2021-01-02,Fc180,0.0,1.0
2,2021-01-02,Fc206,0.0,150.0
3,2021-01-02,Fc251,0.0,300.0
4,2021-01-02,Fc266,0.0,300.0


In [4]:
df_trol = pd.read_csv("df_trolley_to2024.csv",sep=";")
df_trol.head()

,auctiondate,location,crop_type,fpc_1,fpc_2,fpc_3,fpc_4,fpc_5,fpc_6,fpc_7,...,fpc_20,fpc_21,fpc_22,fpc_23,fpc_24,fpc_25,fpc_26,fpc_27,fpc_28,total_trolleys
0,2021-01-04,Aalsmeer,BL,287,89,94,0,0,126,0,...,0,0,3481,1525,51,112,21,989,0,8964
1,2021-01-04,Aalsmeer,PL,0,0,0,9,550,0,54,...,261,120,0,0,0,0,0,0,22,2010
2,2021-01-04,Naaldwijk,BL,106,55,34,0,0,60,0,...,0,0,942,783,8,56,0,214,0,4150
3,2021-01-04,Naaldwijk,PL,0,0,0,5,444,0,45,...,216,92,0,0,0,0,0,0,9,1523
4,2021-01-04,Rijnsburg,BL,51,59,5,1,60,72,3,...,24,4,570,670,5,5,1,436,0,2491


In [5]:
df_fust_new = (
    df_fust
    .groupby("Date", as_index=False)
    .agg(
        total_issued=("number_issued", "sum"),
        total_returned=("number_received", "sum")
    )
)

In [6]:
df_trol["location"] = df_trol["location"].astype("category")
df_trol["crop_type"] = df_trol["crop_type"].astype("category")

df_trol_new = (
    df_trol
    .drop(columns=["total_trolleys"])
    .melt(
        id_vars=df_trol.columns[:3],
        var_name="fpc_code",
        value_name="trolleys"
    )
)

In [7]:
df_dir = df_dir[df_dir.drop(columns=["logistieke location id"]).notna().all(axis=1)]

In [8]:
df_auc_new = (
    df_auc
    .drop(columns=[
        "cropgroupcode_coded",
        "productgroupcode_coded",
        "country of origin iso",
        "avg_price_per_item"
    ])
    .groupby(
        ["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        as_index=False
    )
    .agg(
        total_transactions=("total_transactions", "sum"),
        total_items_sold=("total_items_sold", "sum"),
        total_package=("total_package", "sum"),
        total_revenue=("total_revenue", "sum")
    )
)

df_auc_new["avg_price_per_item"] = (
    df_auc_new["total_revenue"] / df_auc_new["total_items_sold"]
).round(2)

In [9]:
df_auc_new = (
    df_auc_new
    .melt(
        id_vars=["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        value_vars=[
            "total_transactions",
            "total_items_sold",
            "total_package",
            "total_revenue",
            "avg_price_per_item"
        ],
        var_name="var",
        value_name="value"
    )
    .pivot_table(
        index=["date", "logistieke location id", "crop_type", "fpc_index"],
        columns=["sales value id", "var"],
        values="value"
    )
)

df_auc_new.columns = [
    f"{col[0]}_{col[1]}" for col in df_auc_new.columns
]
df_auc_new = df_auc_new.reset_index()

df_auc_new["logistieke location id"] = df_auc_new["logistieke location id"].replace({
    1: "Aalsmeer",
    2: "Naaldwijk",
    3: "Rijnsburg"
}).astype("category")

numeric_cols_auc = df_auc_new.select_dtypes(include="number").columns
df_auc_new[numeric_cols_auc] = df_auc_new[numeric_cols_auc].fillna(0)
df_auc_new = df_auc_new.dropna()

In [10]:
df_dir_3 = df_dir[df_dir["sales value id"] == 3]
df_dir_4 = df_dir[df_dir["sales value id"] == 4]

In [11]:
df_dir_3_new = (
    df_dir_3
    .drop(columns=[
        "cropgroupcode_coded",
        "productgroupcode_coded",
        "country of origin iso",
        "avg_price_per_item"
    ])
    .groupby(
        ["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        as_index=False
    )
    .agg(
        total_transactions=("total_transactions", "sum"),
        total_items_sold=("total_items_sold", "sum"),
        total_package=("total_package", "sum"),
        total_revenue=("total_revenue", "sum")
    )
)

df_dir_3_new["avg_price_per_item"] = (
    df_dir_3_new["total_revenue"] / df_dir_3_new["total_items_sold"]
).round(2)

In [12]:
df_dir_3_new = (
    df_dir_3_new
    .melt(
        id_vars=["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        value_vars=[
            "total_transactions",
            "total_items_sold",
            "total_package",
            "total_revenue",
            "avg_price_per_item"
        ],
        var_name="var",
        value_name="value"
    )
    .pivot_table(
        index=["date", "logistieke location id", "crop_type", "fpc_index"],
        columns=["sales value id", "var"],
        values="value"
    )
)

df_dir_3_new.columns = [
    f"{col[0]}_{col[1]}" for col in df_dir_3_new.columns
]
df_dir_3_new = df_dir_3_new.reset_index()

In [13]:
df_dir_4_new = (
    df_dir_4
    .drop(columns=[
        "cropgroupcode_coded",
        "productgroupcode_coded",
        "country of origin iso",
        "avg_price_per_item",
        "logistieke location id"
    ])
    .groupby(
        ["date", "crop_type", "fpc_index", "sales value id"],
        as_index=False
    )
    .agg(
        total_transactions=("total_transactions", "sum"),
        total_items_sold=("total_items_sold", "sum"),
        total_package=("total_package", "sum"),
        total_revenue=("total_revenue", "sum")
    )
)

df_dir_4_new["avg_price_per_item"] = (
    df_dir_4_new["total_revenue"] / df_dir_4_new["total_items_sold"]
).round(2)

In [14]:
df_dir_4_new = (
    df_dir_4_new
    .melt(
        id_vars=["date", "crop_type", "fpc_index", "sales value id"],
        value_vars=[
            "total_transactions",
            "total_items_sold",
            "total_package",
            "total_revenue",
            "avg_price_per_item"
        ],
        var_name="var",
        value_name="value"
    )
    .pivot_table(
        index=["date", "crop_type", "fpc_index"],
        columns=["sales value id", "var"],
        values="value"
    )
)

df_dir_4_new.columns = [
    f"{col[0]}_{col[1]}" for col in df_dir_4_new.columns
]
df_dir_4_new = df_dir_4_new.reset_index()

In [15]:
df_dir_new = df_dir_3_new.merge(
    df_dir_4_new,
    on=["date", "crop_type", "fpc_index"],
    how="outer"
)

df_dir_new["logistieke location id"] = df_dir_new["logistieke location id"].replace({
    1: "Aalsmeer",
    2: "Naaldwijk",
    3: "Rijnsburg"
}).astype("category")

numeric_cols_dir = df_dir_new.select_dtypes(include="number").columns
df_dir_new[numeric_cols_dir] = df_dir_new[numeric_cols_dir].fillna(0)

df_dir_new = df_dir_new.dropna()

In [16]:
new1 = df_dir_new.merge(
    df_auc_new,
    on=["date", "crop_type", "fpc_index", "logistieke location id"],
    how="outer"
)

numeric_cols_new1 = new1.select_dtypes(include="number").columns
new1[numeric_cols_new1] = new1[numeric_cols_new1].fillna(0)

new1 = new1.dropna()

In [17]:
new2 = new1.merge(
    df_trol_new,
    left_on=["date", "crop_type", "fpc_index", "logistieke location id"],
    right_on=["auctiondate", "crop_type", "fpc_code", "location"],
    how="inner"
)

In [18]:
merged_df = new2.merge(
    df_fust_new,
    left_on="date",
    right_on="Date",
    how="left"
)

In [19]:
merged_df = merged_df.drop(columns=["Date", "auctiondate", "location", "fpc_code"])
merged_df = merged_df.dropna()
merged_df.head()

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,1_total_revenue,1_total_transactions,5_avg_price_per_item,5_total_items_sold,5_total_package,5_total_revenue,5_total_transactions,trolleys,total_issued,total_returned
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,18960.20,717.0,0.24,312850.0,4770.0,76058.00,1237.0,287,621375.0,771606.0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,5483.50,227.0,0.26,110970.0,1762.0,29005.80,553.0,106,621375.0,771606.0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,4349.40,167.0,0.26,50810.0,786.0,13353.90,302.0,51,621375.0,771606.0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,18427.65,674.0,0.20,356840.0,4085.0,71239.65,1114.0,191,621375.0,771606.0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,7674.80,292.0,0.19,239939.0,2952.0,45257.18,918.0,138,621375.0,771606.0


In [20]:
merged_df.shape[0]

72080

In [21]:
#Add year, month, weekday(0=Monday, 6=Sunday), weekofyear (1=Jan, 12=Dec) columns to the merged dataframe. 

In [22]:
merged_df["date"] = pd.to_datetime(merged_df["date"])

merged_df = merged_df.assign(
    year=merged_df["date"].dt.year,
    month=merged_df["date"].dt.month,
    weekday=merged_df["date"].dt.weekday,
    weekofyear=merged_df["date"].dt.isocalendar().week
)
merged_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,5_total_package,5_total_revenue,5_total_transactions,trolleys,total_issued,total_returned,year,month,weekday,weekofyear
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,4770.0,76058.00,1237.0,287,621375.0,771606.0,2021,1,0,1
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,1762.0,29005.80,553.0,106,621375.0,771606.0,2021,1,0,1
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,786.0,13353.90,302.0,51,621375.0,771606.0,2021,1,0,1
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,4085.0,71239.65,1114.0,191,621375.0,771606.0,2021,1,0,1
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,2952.0,45257.18,918.0,138,621375.0,771606.0,2021,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,13084.0,107720.50,1113.0,357,623400.0,1060519.0,2024,12,0,1
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,1940.0,17491.30,177.0,57,623400.0,1060519.0,2024,12,0,1
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,1033.0,7516.67,110.0,29,623400.0,1060519.0,2024,12,0,1
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,793.0,14334.65,115.0,32,623400.0,1060519.0,2024,12,0,1


In [23]:
#Add a binary variable column is_busy_period Feb-May (1=date is within the busy period, 0=date is outside of the busy period).

In [24]:
merged_df["is_busy_period"] = merged_df["month"].isin([2, 3, 4, 5]).astype(int)

merged_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,5_total_revenue,5_total_transactions,trolleys,total_issued,total_returned,year,month,weekday,weekofyear,is_busy_period
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,76058.00,1237.0,287,621375.0,771606.0,2021,1,0,1,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,29005.80,553.0,106,621375.0,771606.0,2021,1,0,1,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,13353.90,302.0,51,621375.0,771606.0,2021,1,0,1,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,71239.65,1114.0,191,621375.0,771606.0,2021,1,0,1,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,45257.18,918.0,138,621375.0,771606.0,2021,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,107720.50,1113.0,357,623400.0,1060519.0,2024,12,0,1,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,17491.30,177.0,57,623400.0,1060519.0,2024,12,0,1,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,7516.67,110.0,29,623400.0,1060519.0,2024,12,0,1,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,14334.65,115.0,32,623400.0,1060519.0,2024,12,0,1,0


In [25]:
#Create a holidays data frame. 

In [26]:
holidays_df=merged_df

In [27]:
##Add a column is_valentine_window to highlight the valentine's day window: 14th Feb and 3 weeks before it. 
#1:date is within Valentine's day window, 0:date is outside Valentine's day window.

In [28]:
valentine = pd.to_datetime(merged_df["year"].astype(str) + "-02-14")
holidays_df["is_valentine_window"] = (
    (merged_df["date"] >= (valentine - pd.Timedelta(days=21))) &
    (merged_df["date"] <= valentine)
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,5_total_transactions,trolleys,total_issued,total_returned,year,month,weekday,weekofyear,is_busy_period,is_valentine_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,1237.0,287,621375.0,771606.0,2021,1,0,1,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,553.0,106,621375.0,771606.0,2021,1,0,1,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,302.0,51,621375.0,771606.0,2021,1,0,1,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,1114.0,191,621375.0,771606.0,2021,1,0,1,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,918.0,138,621375.0,771606.0,2021,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,1113.0,357,623400.0,1060519.0,2024,12,0,1,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,177.0,57,623400.0,1060519.0,2024,12,0,1,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,110.0,29,623400.0,1060519.0,2024,12,0,1,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,115.0,32,623400.0,1060519.0,2024,12,0,1,0,0


In [29]:
(holidays_df["date"] == pd.Timestamp("2021-02-14")).sum()

np.int64(0)

In [30]:
(holidays_df["date"] == pd.Timestamp("2022-02-14")).sum()

np.int64(70)

In [31]:
(holidays_df["date"] == pd.Timestamp("2023-02-14")).sum()

np.int64(71)

In [32]:
(holidays_df["date"] == pd.Timestamp("2024-02-14")).sum()

np.int64(70)

In [33]:
#Add a column is_mothersday_nl_window to highlight the mother's day in NL window: 2nd Sunday of May and 1 week before it. 
#1:date is within Mother's day in NL window, 0:date is outside Mother's day in NL window.

In [34]:
def mothers_day_nl(year: int) -> pd.Timestamp:
    # all days in May
    may = pd.date_range(f"{year}-05-01", f"{year}-05-31", freq="D")
    # Sundays (Monday=0 ... Sunday=6)
    sundays = may[may.weekday == 6]
    # 2nd Sunday
    return sundays[1]
    
holidays_df["date"] = pd.to_datetime(holidays_df["date"]).dt.normalize()

# Compute Mother's Day date for each row's year
md_nl = holidays_df["year"].map(lambda y: mothers_day_nl(int(y)))

# Window: 7 days before + Mother's Day
holidays_df["is_mothersday_nl_window"] = (
    (holidays_df["date"] >= (md_nl - pd.Timedelta(days=7))) &
    (holidays_df["date"] <= md_nl)
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,trolleys,total_issued,total_returned,year,month,weekday,weekofyear,is_busy_period,is_valentine_window,is_mothersday_nl_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,287,621375.0,771606.0,2021,1,0,1,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,106,621375.0,771606.0,2021,1,0,1,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,51,621375.0,771606.0,2021,1,0,1,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,191,621375.0,771606.0,2021,1,0,1,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,138,621375.0,771606.0,2021,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,357,623400.0,1060519.0,2024,12,0,1,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,57,623400.0,1060519.0,2024,12,0,1,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,29,623400.0,1060519.0,2024,12,0,1,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,32,623400.0,1060519.0,2024,12,0,1,0,0,0


In [35]:
#Add a column is_int_womensday_window to highlight the International Wome's Day window: 8th of March and 2 weeks before it. 
#1:date is within International Women's Day window, 0:date is outside International Women's Day window.

In [36]:
# March 8 of each row's year
womens_day = pd.to_datetime(holidays_df["year"].astype(str) + "-03-08")

# International Women’s Day: 2 weeks before up to March 8
holidays_df["is_int_womensday_window"] = (
    (holidays_df["date"] >= (womens_day - pd.Timedelta(days=14))) &
    (holidays_df["date"] <= womens_day)
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,total_issued,total_returned,year,month,weekday,weekofyear,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,621375.0,771606.0,2021,1,0,1,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,621375.0,771606.0,2021,1,0,1,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,621375.0,771606.0,2021,1,0,1,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,621375.0,771606.0,2021,1,0,1,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,621375.0,771606.0,2021,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,623400.0,1060519.0,2024,12,0,1,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,623400.0,1060519.0,2024,12,0,1,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,623400.0,1060519.0,2024,12,0,1,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,623400.0,1060519.0,2024,12,0,1,0,0,0,0


In [37]:
#Add a column is_rus_womensday_window to highlight the Women's Day in Russia window: 8th of March and 1-2 weeks before it (we take an average: 10 days).
#1:date is within Women's Day in Russia window, 0:date is outside Women's Day in Russia window.

In [38]:
# Women’s Day in Russia window: 10 days before up to March 8
holidays_df["is_rus_womensday_window"] = (
    (holidays_df["date"] >= (womens_day - pd.Timedelta(days=10))) &
    (holidays_df["date"] <= womens_day)
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,total_returned,year,month,weekday,weekofyear,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,771606.0,2021,1,0,1,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,771606.0,2021,1,0,1,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,771606.0,2021,1,0,1,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,771606.0,2021,1,0,1,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,771606.0,2021,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,1060519.0,2024,12,0,1,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,1060519.0,2024,12,0,1,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,1060519.0,2024,12,0,1,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,1060519.0,2024,12,0,1,0,0,0,0,0


In [39]:
#Add a column is_kingsday_window to highlight the King's Day window: 27th of April, 1 day before and after it.
#1:date is within King's Day window, 0:date is outside King's Day window.

In [40]:
# Create King's Day date for each row's year
kings_day = pd.to_datetime(holidays_df["date"].dt.year.astype(str) + "-04-27")

# window: 1 day before and 1 day after
holidays_df["is_kingsday_window"] = (
    (holidays_df["date"] >= (kings_day - pd.Timedelta(days=1))) &
    (holidays_df["date"] <= (kings_day + pd.Timedelta(days=1)))
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,year,month,weekday,weekofyear,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,2021,1,0,1,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,2021,1,0,1,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,2021,1,0,1,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,2021,1,0,1,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,2021,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,2024,12,0,1,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,2024,12,0,1,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,2024,12,0,1,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,2024,12,0,1,0,0,0,0,0,0


In [41]:
#Add a column is_christmas_window to highlight the Christmas window: 25th of December, 2 days before and 3 days after it.
#1:date is within Christmas window, 0:date is outside Christmas window.

In [42]:
# Create Christmas date for each row's year
christmas= pd.to_datetime(holidays_df["date"].dt.year.astype(str) + "-12-25")

# window: 2 days before and 3 days after
holidays_df["is_christmas_window"] = (
    (holidays_df["date"] >= (christmas - pd.Timedelta(days=2))) &
    (holidays_df["date"] <= (christmas + pd.Timedelta(days=3)))
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,month,weekday,weekofyear,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,1,0,1,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,1,0,1,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,1,0,1,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,1,0,1,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,12,0,1,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,12,0,1,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,12,0,1,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,12,0,1,0,0,0,0,0,0,0


In [43]:
#Add a column is_newyear_window to highlight the New Year's window: 1st of January and 1 day after it.
#1:date is within New Year's window, 0:date is outside New Year's window.

In [44]:
# Create New Year's date for each row's year
new_year= pd.to_datetime(holidays_df["date"].dt.year.astype(str) + "-01-01")

# window: 1 day after
holidays_df["is_newyear_window"] = (
    (holidays_df["date"] >= new_year ) &
    (holidays_df["date"] <= (new_year + pd.Timedelta(days=1)))
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,weekday,weekofyear,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,0,1,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,1,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,1,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,0,1,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,0,1,0,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,0,1,0,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,0,1,0,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,0,1,0,0,0,0,0,0,0,0


In [45]:
#Add a column is_allsoulsday_window to highlight the All Soul's Day window: 2nd of November and 10 days before it.
#1:date is within All Soul's Day window, 0:date is outside All Soul's Day window.

In [46]:
# Create All Soul's Day date for each row's year
allsouls_day= pd.to_datetime(holidays_df["date"].dt.year.astype(str) + "-11-02")

# window: 10 days before
holidays_df["is_allsoulsday_window"] = (
    (holidays_df["date"] >= (allsouls_day  - pd.Timedelta(days=2))) &
    (holidays_df["date"] <= allsouls_day)
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,weekofyear,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,1,0,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,1,0,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,1,0,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,1,0,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,1,0,0,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,1,0,0,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,1,0,0,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,1,0,0,0,0,0,0,0,0,0


In [47]:
#Create an Easter dataframe with Easter Sunday and Easter Monday dates throughout 2021-2024.

In [48]:
easter_df = pd.DataFrame({
    "year": [2021, 2022, 2023, 2024],
    "easter_sunday": pd.to_datetime([
        "2021-04-04", "2022-04-17", "2023-04-09", "2024-03-31"
    ]),
    "easter_monday": pd.to_datetime([
        "2021-04-05", "2022-04-18", "2023-04-10", "2024-04-01"
    ])})

easter_df

,year,easter_sunday,easter_monday
0,2021,2021-04-04,2021-04-05
1,2022,2022-04-17,2022-04-18
2,2023,2023-04-09,2023-04-10
3,2024,2024-03-31,2024-04-01


In [49]:
#Add a column is_easter_monday_window to highlight the Easter Monday's wundow: Friday before Easter Monday (3 days before) and 2 days after.
#1:date is within Easter Monday window, 0:date is outside Easter Monday window.

In [50]:
easter_monday_lookup = easter_df.set_index("year")["easter_monday"]
easter_monday_for_row = holidays_df["year"].map(easter_monday_lookup)

holidays_df["is_easter_monday_window"] = (
    (holidays_df["date"] >= easter_monday_for_row - pd.Timedelta(days=3)) &
    (holidays_df["date"] <= easter_monday_for_row + pd.Timedelta(days=2))
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,0,0,0,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,0,0,0,0,0,0,0,0,0,0


In [51]:
(holidays_df["date"] == pd.Timestamp("2021-04-07")).sum()

np.int64(73)

In [52]:
#2021-04-07 falls under Easter Monday window, and we check if it is 1 for is_easter_monday_window.

In [53]:
holidays_df.loc[
    holidays_df["date"] == pd.Timestamp("2021-04-07")
]

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_busy_period,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window
4733,2021-04-07,Aalsmeer,BL,fpc_1,0.22,86920.0,1058.0,18945.50,101.0,0.23,...,1,0,0,0,0,0,0,0,0,1
4734,2021-04-07,Naaldwijk,BL,fpc_1,0.25,54500.0,706.0,13760.80,68.0,0.23,...,1,0,0,0,0,0,0,0,0,1
4735,2021-04-07,Rijnsburg,BL,fpc_1,0.18,42250.0,475.0,7709.50,69.0,0.23,...,1,0,0,0,0,0,0,0,0,1
4736,2021-04-07,Aalsmeer,BL,fpc_10,0.28,15285.0,165.0,4229.00,29.0,0.26,...,1,0,0,0,0,0,0,0,0,1
4737,2021-04-07,Naaldwijk,BL,fpc_10,0.37,5549.0,104.0,2045.30,30.0,0.26,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4801,2021-04-07,Naaldwijk,PL,fpc_5,1.94,124448.0,17842.0,241972.13,2675.0,1.83,...,1,0,0,0,0,0,0,0,0,1
4802,2021-04-07,Aalsmeer,PL,fpc_7,2.18,27254.0,5494.0,59427.03,402.0,2.36,...,1,0,0,0,0,0,0,0,0,1
4803,2021-04-07,Naaldwijk,PL,fpc_7,2.28,45344.0,10155.0,103463.44,554.0,2.36,...,1,0,0,0,0,0,0,0,0,1
4804,2021-04-07,Aalsmeer,PL,fpc_8,2.81,11103.0,1545.0,31182.20,223.0,2.37,...,1,0,0,0,0,0,0,0,0,1


In [54]:
#Create a column in easter_df for Whit Monday dates (Easter Sunday+50 days).

In [55]:
# Whit Monday = Easter Sunday + 50 days
easter_df["whit_monday"] = easter_df["easter_sunday"] + pd.Timedelta(days=50)

easter_df

,year,easter_sunday,easter_monday,whit_monday
0,2021,2021-04-04,2021-04-05,2021-05-24
1,2022,2022-04-17,2022-04-18,2022-06-06
2,2023,2023-04-09,2023-04-10,2023-05-29
3,2024,2024-03-31,2024-04-01,2024-05-20


In [56]:
#Add a column is_whit_monday_window to highlight the Whit Monday's window: Friday before Whit Monday (3 days before) and 2 days after.
#1:date is within Whit Monday window, 0:date is outside Whit Monday window.

In [57]:
whit_lookup = easter_df.set_index("year")["whit_monday"]
whit_monday_for_row = holidays_df["year"].map(whit_lookup)

holidays_df["is_whit_monday_window"] = (
    (holidays_df["date"] >= whit_monday_for_row - pd.Timedelta(days=3)) &
    (holidays_df["date"] <= whit_monday_for_row + pd.Timedelta(days=2))
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,0,0,0,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,0,0,0,0,0,0,0,0,0,0


In [58]:
#2021-05-25 falls under Whit Monday window, and we check if it is 1 for is_whit_monday_window.

In [59]:
holidays_df.loc[
    holidays_df["date"] == pd.Timestamp("2021-05-25")
]

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window
6991,2021-05-25,Aalsmeer,BL,fpc_1,0.34,110770.0,1695.0,38079.50,119.0,0.28,...,0,0,0,0,0,0,0,0,0,1
6992,2021-05-25,Naaldwijk,BL,fpc_1,0.39,34390.0,547.0,13298.30,85.0,0.28,...,0,0,0,0,0,0,0,0,0,1
6993,2021-05-25,Rijnsburg,BL,fpc_1,0.30,117180.0,1574.0,35238.90,123.0,0.28,...,0,0,0,0,0,0,0,0,0,1
6994,2021-05-25,Aalsmeer,BL,fpc_10,0.29,51361.0,593.0,15019.30,64.0,0.26,...,0,0,0,0,0,0,0,0,0,1
6995,2021-05-25,Naaldwijk,BL,fpc_10,0.26,21982.0,235.0,5605.47,61.0,0.26,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7059,2021-05-25,Naaldwijk,PL,fpc_5,2.07,98654.0,15634.0,204089.27,1323.0,2.26,...,0,0,0,0,0,0,0,0,0,1
7060,2021-05-25,Aalsmeer,PL,fpc_7,2.14,21617.0,5449.0,46273.73,160.0,2.31,...,0,0,0,0,0,0,0,0,0,1
7061,2021-05-25,Naaldwijk,PL,fpc_7,2.48,39395.0,10523.0,97664.08,436.0,2.31,...,0,0,0,0,0,0,0,0,0,1
7062,2021-05-25,Aalsmeer,PL,fpc_8,3.04,8660.0,1152.0,26356.22,138.0,2.80,...,0,0,0,0,0,0,0,0,0,1


In [60]:
#Create a column in easter_df for Ascension Day dates (Easter Sunday+40 days).

In [61]:
# Ascension Day = Easter Sunday + 40 days
easter_df["ascension_day"] = easter_df["easter_sunday"] + pd.Timedelta(days=40)

easter_df

,year,easter_sunday,easter_monday,whit_monday,ascension_day
0,2021,2021-04-04,2021-04-05,2021-05-24,2021-05-14
1,2022,2022-04-17,2022-04-18,2022-06-06,2022-05-27
2,2023,2023-04-09,2023-04-10,2023-05-29,2023-05-19
3,2024,2024-03-31,2024-04-01,2024-05-20,2024-05-10


In [62]:
holidays_df.loc[
    holidays_df["date"] == pd.Timestamp("2023-05-18")
]

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_valentine_window,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window


In [63]:
#Add a column is_ascension_day_window to highlight the Ascension Day's window: Ascension day and 1 day after.
#1:date is within Ascension Day window, 0:date is outside Ascension Day window.

In [64]:
ascension_lookup = easter_df.set_index("year")["ascension_day"]
ascension_for_row = holidays_df["year"].map(ascension_lookup)

holidays_df["is_ascension_day_window"] = (
    (holidays_df["date"] >= ascension_for_row) &
    (holidays_df["date"] <= ascension_for_row + pd.Timedelta(days=1))
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,0,0,0,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,0,0,0,0,0,0,0,0,0,0


In [65]:
holidays_df.loc[
    holidays_df["date"] == pd.Timestamp("2023-05-19")
]

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_mothersday_nl_window,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window
43172,2023-05-19,Aalsmeer,BL,fpc_1,0.20,53230.0,626.0,10803.40,61.0,0.28,...,0,0,0,0,0,0,0,0,0,1
43173,2023-05-19,Naaldwijk,BL,fpc_1,0.28,12640.0,155.0,3534.80,22.0,0.28,...,0,0,0,0,0,0,0,0,0,1
43174,2023-05-19,Rijnsburg,BL,fpc_1,0.31,12230.0,191.0,3746.70,34.0,0.28,...,0,0,0,0,0,0,0,0,0,1
43175,2023-05-19,Aalsmeer,BL,fpc_10,0.30,28680.0,463.0,8517.40,67.0,0.17,...,0,0,0,0,0,0,0,0,0,1
43176,2023-05-19,Naaldwijk,BL,fpc_10,0.33,11041.0,163.0,3629.80,44.0,0.17,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43239,2023-05-19,Naaldwijk,PL,fpc_5,1.89,88255.0,13294.0,166938.39,1536.0,2.05,...,0,0,0,0,0,0,0,0,0,1
43240,2023-05-19,Aalsmeer,PL,fpc_7,2.55,34090.0,10005.0,86983.97,409.0,2.30,...,0,0,0,0,0,0,0,0,0,1
43241,2023-05-19,Naaldwijk,PL,fpc_7,2.58,41875.0,11351.0,107941.50,834.0,2.30,...,0,0,0,0,0,0,0,0,0,1
43242,2023-05-19,Aalsmeer,PL,fpc_8,2.74,7357.0,889.0,20172.76,117.0,2.13,...,0,0,0,0,0,0,0,0,0,1


In [66]:
#Add a column is_mothersday_fr_window to highlight the mother's day in FR window: last Sunday of May and 1 week before it. 
#Note: if last Sunday of May falls under Whit Sunday (easter sunday+49 days), mother's day is moved to the first Sunday of June.
#1:date is within Mother's day in FR window, 0:date is outside Mother's day in FR window.

In [67]:
# year -> Easter Sunday lookup
easter_lookup = easter_df.set_index("year")["easter_sunday"]

def mothers_day_fr(year: int) -> pd.Timestamp:
    # last Sunday of May
    may = pd.date_range(f"{year}-05-01", f"{year}-05-31", freq="D")
    last_sunday_may = may[may.weekday == 6][-1]

    # Whit Sunday = Easter Sunday + 49 days (from easter_df)
    whit_sunday = easter_lookup.loc[year] + pd.Timedelta(days=49)

    # if conflict, move to first Sunday of June
    if last_sunday_may.normalize() == whit_sunday.normalize():
        june = pd.date_range(f"{year}-06-01", f"{year}-06-30", freq="D")
        return june[june.weekday == 6][0]

    return last_sunday_may

# per-row Mother's Day France date
md_fr = holidays_df["year"].map(lambda y: mothers_day_fr(int(y)))

# 1 week before window flag
holidays_df["is_mothersday_fr_window"] = (
    (holidays_df["date"] >= md_fr - pd.Timedelta(days=7)) &
    (holidays_df["date"] <= md_fr)
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window,is_mothersday_fr_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,0,0,0,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,0,0,0,0,0,0,0,0,0,0


In [68]:
holidays_df.loc[
    holidays_df["date"] == pd.Timestamp("2021-05-28")
]

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_int_womensday_window,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window,is_mothersday_fr_window
7209,2021-05-28,Aalsmeer,BL,fpc_1,0.21,37160.0,456.0,7828.40,47.0,0.26,...,0,0,0,0,0,0,0,0,0,1
7210,2021-05-28,Naaldwijk,BL,fpc_1,0.24,4390.0,55.0,1075.20,9.0,0.26,...,0,0,0,0,0,0,0,0,0,1
7211,2021-05-28,Rijnsburg,BL,fpc_1,0.27,19230.0,251.0,5151.80,33.0,0.26,...,0,0,0,0,0,0,0,0,0,1
7212,2021-05-28,Aalsmeer,BL,fpc_10,0.35,12976.0,138.0,4521.20,28.0,0.50,...,0,0,0,0,0,0,0,0,0,1
7213,2021-05-28,Naaldwijk,BL,fpc_10,0.32,4551.0,64.0,1471.60,25.0,0.50,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7276,2021-05-28,Naaldwijk,PL,fpc_5,1.94,58506.0,8560.0,113407.56,971.0,1.80,...,0,0,0,0,0,0,0,0,0,1
7277,2021-05-28,Aalsmeer,PL,fpc_7,1.87,18778.0,4943.0,35155.04,154.0,2.21,...,0,0,0,0,0,0,0,0,0,1
7278,2021-05-28,Naaldwijk,PL,fpc_7,2.54,26787.0,7128.0,68083.37,331.0,2.21,...,0,0,0,0,0,0,0,0,0,1
7279,2021-05-28,Aalsmeer,PL,fpc_8,3.10,5826.0,1209.0,18038.92,101.0,2.98,...,0,0,0,0,0,0,0,0,0,1


In [69]:
#Add a column is_mothersday_uk_window to highlight the mother's day in UK window: (Mother’s Day (UK)=Easter Sunday−21 days) and 1 week before it. 
#1:date is within Mother's day in UK window, 0:date is outside Mother's day in UK window.

In [70]:
# year -> Easter Sunday lookup
easter_lookup = easter_df.set_index("year")["easter_sunday"]

def mothers_day_uk(year: int) -> pd.Timestamp:
    # Mothering Sunday = Easter Sunday - 21 days
    return easter_lookup.loc[year] - pd.Timedelta(days=21)

# per-row UK Mother's Day date
md_uk = holidays_df["year"].map(lambda y: mothers_day_uk(int(y)))

# 1 week before window
holidays_df["is_mothersday_uk_window"] = (
    (holidays_df["date"] >= md_uk - pd.Timedelta(days=7)) &
    (holidays_df["date"] <= md_uk)
).astype(int)

holidays_df

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window,is_mothersday_fr_window,is_mothersday_uk_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72075,2024-12-30,Naaldwijk,PL,fpc_5,1.61,59773.0,7133.0,96403.69,1410.0,1.28,...,0,0,0,0,0,0,0,0,0,0
72076,2024-12-30,Aalsmeer,PL,fpc_7,1.63,2982.0,470.0,4862.88,32.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72077,2024-12-30,Naaldwijk,PL,fpc_7,2.84,7675.0,1024.0,21776.71,137.0,1.32,...,0,0,0,0,0,0,0,0,0,0
72078,2024-12-30,Aalsmeer,PL,fpc_8,2.58,8885.0,819.0,22943.19,89.0,1.77,...,0,0,0,0,0,0,0,0,0,0


In [71]:
holidays_df.loc[
    holidays_df["date"] == pd.Timestamp("2021-03-12")
]

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window,is_mothersday_fr_window,is_mothersday_uk_window
3510,2021-03-12,Aalsmeer,BL,fpc_1,0.37,11740.0,171.0,4350.70,24.0,0.32,...,0,0,0,0,0,0,0,0,0,1
3511,2021-03-12,Naaldwijk,BL,fpc_1,0.29,12310.0,144.0,3596.40,24.0,0.32,...,0,0,0,0,0,0,0,0,0,1
3512,2021-03-12,Rijnsburg,BL,fpc_1,0.30,20590.0,232.0,6143.00,43.0,0.32,...,0,0,0,0,0,0,0,0,0,1
3513,2021-03-12,Aalsmeer,BL,fpc_10,0.22,38360.0,770.0,8261.56,20.0,0.38,...,0,0,0,0,0,0,0,0,0,1
3514,2021-03-12,Naaldwijk,BL,fpc_10,0.40,3898.0,70.0,1556.09,21.0,0.38,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577,2021-03-12,Naaldwijk,PL,fpc_5,1.50,98635.0,11342.0,147498.12,1349.0,1.37,...,0,0,0,0,0,0,0,0,0,1
3578,2021-03-12,Aalsmeer,PL,fpc_7,1.37,19816.0,3337.0,27092.30,137.0,2.34,...,0,0,0,0,0,0,0,0,0,1
3579,2021-03-12,Naaldwijk,PL,fpc_7,2.23,26787.0,4793.0,59862.81,334.0,2.34,...,0,0,0,0,0,0,0,0,0,1
3580,2021-03-12,Aalsmeer,PL,fpc_8,2.98,12378.0,1665.0,36850.40,210.0,2.49,...,0,0,0,0,0,0,0,0,0,1


In [72]:
#Create a data frame where only holidays window dates are included.

In [73]:
holiday_cols = [
    "is_busy_period",
    "is_valentine_window",
    "is_mothersday_nl_window",
    "is_int_womensday_window",
    "is_rus_womensday_window",
    "is_kingsday_window",
    "is_christmas_window",
    "is_newyear_window",
    "is_allsoulsday_window",
    "is_easter_monday_window",
    "is_whit_monday_window",
    "is_ascension_day_window",
    "is_mothersday_fr_window",
    "is_mothersday_uk_window"
]

In [74]:
holidays_only_df = holidays_df[
    holidays_df[holiday_cols].sum(axis=1) > 0
].copy()

holidays_only_df


,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window,is_mothersday_fr_window,is_mothersday_uk_window
1063,2021-01-25,Aalsmeer,BL,fpc_1,0.24,40410.0,514.0,9552.70,84.0,0.31,...,0,0,0,0,0,0,0,0,0,0
1064,2021-01-25,Naaldwijk,BL,fpc_1,0.29,18000.0,274.0,5291.00,51.0,0.31,...,0,0,0,0,0,0,0,0,0,0
1065,2021-01-25,Rijnsburg,BL,fpc_1,0.24,22770.0,300.0,5448.80,49.0,0.31,...,0,0,0,0,0,0,0,0,0,0
1066,2021-01-25,Aalsmeer,BL,fpc_10,0.15,90479.0,570.0,13984.70,32.0,0.31,...,0,0,0,0,0,0,0,0,0,0
1067,2021-01-25,Naaldwijk,BL,fpc_10,0.34,5501.0,85.0,1844.40,20.0,0.31,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72007,2024-12-27,Naaldwijk,PL,fpc_5,1.48,23860.0,3002.0,35246.49,335.0,1.17,...,0,0,1,0,0,0,0,0,0,0
72008,2024-12-27,Aalsmeer,PL,fpc_7,1.04,2322.0,301.0,2420.58,10.0,0.91,...,0,0,1,0,0,0,0,0,0,0
72009,2024-12-27,Naaldwijk,PL,fpc_7,3.42,718.0,110.0,2457.77,30.0,0.91,...,0,0,1,0,0,0,0,0,0,0
72010,2024-12-27,Aalsmeer,PL,fpc_8,3.55,2578.0,300.0,9163.26,39.0,1.30,...,0,0,1,0,0,0,0,0,0,0


In [75]:
#merged_df is the data frame where all dates are included, holidays are not flagged. Busy period is included.
#holidays_df is the data frame where all dates are included and holidays are flagged. Busy period is also included.
#holidays_only_df is the data frame where only holidays windows dates are included. Busy period is also included.

In [76]:
merged_df.to_csv("merged_df.csv", index=False)
holidays_df.to_csv("holidays_df.csv", index=False)
holidays_only_df.to_csv("holidays_only_df.csv", index=False)

In [77]:
import pandas as pd

holidays_df = pd.read_csv("holidays_df.csv", sep=",")
holidays_df.head()

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,is_rus_womensday_window,is_kingsday_window,is_christmas_window,is_newyear_window,is_allsoulsday_window,is_easter_monday_window,is_whit_monday_window,is_ascension_day_window,is_mothersday_fr_window,is_mothersday_uk_window
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,0,0,0,0,0,0,0,0,0,0


In [78]:
holidays_df["date"] = pd.to_datetime(holidays_df["date"])

In [79]:
holidays_df = holidays_df.sort_values(["logistieke location id", "crop_type", "fpc_index", "date"])

In [80]:
Z = holidays_df.columns[4:25].tolist()  # replace with your actual columns
lags = [1, 2, 3, 8, 20, 50]

In [81]:
lagged_cols = {}

g = holidays_df.groupby(["logistieke location id", "crop_type", "fpc_index"], observed=True)

for lag in lags:
    shifted = g[Z].shift(lag)
    shifted.columns = [f"{col}_lag_{lag}" for col in Z]
    lagged_cols[lag] = shifted

In [82]:
lagged_df = pd.concat(lagged_cols.values(), axis=1)
holidays_df = pd.concat([holidays_df, lagged_df], axis=1)

In [83]:
holidays_df.head()

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,1_total_items_sold_lag_50,1_total_package_lag_50,1_total_revenue_lag_50,1_total_transactions_lag_50,5_avg_price_per_item_lag_50,5_total_items_sold_lag_50,5_total_package_lag_50,5_total_revenue_lag_50,5_total_transactions_lag_50,trolleys_lag_50
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,2021-01-05,Aalsmeer,BL,fpc_1,0.21,46610.0,496.0,9828.70,63.0,0.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,2021-01-06,Aalsmeer,BL,fpc_1,0.27,26570.0,327.0,7231.00,39.0,0.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,2021-01-07,Aalsmeer,BL,fpc_1,0.24,22570.0,250.0,5437.70,22.0,0.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,2021-01-08,Aalsmeer,BL,fpc_1,0.26,9540.0,122.0,2513.70,19.0,0.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
holidays_df.to_csv("laggedholidays_df.csv", index=False)

In [85]:
holidays_df["date"] = pd.to_datetime(holidays_df["date"])

analysis_df = holidays_df[
    (holidays_df["date"].dt.year >= 2021) &
    (holidays_df["date"].dt.year <= 2023)
]

assessment_df = holidays_df[holidays_df["date"].dt.year == 2024]

In [86]:
analysis_df = holidays_df[
    (holidays_df["date"] >= "2021-01-01") &
    (holidays_df["date"] <= "2023-12-31")
]

assessment_df = holidays_df[
    (holidays_df["date"] >= "2024-01-01") &
    (holidays_df["date"] <= "2024-12-31")
]


In [87]:
analysis_df.to_csv("analysis_df.csv", index=False)
assessment_df.to_csv("assessment_df.csv", index=False)